## Primer inciso que pide elegir activos

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.display import display, Markdown
%matplotlib inline

In [2]:
tickers=['AMZN','WMT','GOOGL','AAPL','JPM']

f_inicial='2020-01-01'
f_final='2024-11-24'

closes=yf.download(tickers,f_inicial,f_final)['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [3]:
closes

Ticker,AAPL,AMZN,GOOGL,JPM,WMT
Date,,,,,
2020-01-02 00:00:00+00:00,72.796028,94.900497,68.264961,122.104614,36.864063
2020-01-03 00:00:00+00:00,72.088280,93.748497,67.907845,120.493279,36.538635
2020-01-06 00:00:00+00:00,72.662697,95.143997,69.717865,120.397446,36.464256
2020-01-07 00:00:00+00:00,72.320976,95.343002,69.583206,118.350639,36.126423
2020-01-08 00:00:00+00:00,73.484344,94.598503,70.078476,119.273872,36.002445
...,...,...,...,...,...
2024-11-18 00:00:00+00:00,228.020004,201.699997,175.300003,245.029999,84.080002
2024-11-19 00:00:00+00:00,228.279999,204.610001,178.119995,243.089996,86.599998
2024-11-20 00:00:00+00:00,229.000000,202.880005,175.979996,240.779999,87.180000


In [4]:
ret = closes.pct_change().dropna()
mu = ret.mean()
mu # vector mu

Ticker
AAPL     0.001136
AMZN     0.000852
GOOGL    0.000925
JPM      0.000789
WMT      0.000830
dtype: float64

In [5]:
sigma = ret.cov()
sigma # matriz de covarianza

Ticker,AAPL,AMZN,GOOGL,JPM,WMT
Ticker,,,,,
AAPL,0.000404,0.000271,0.000269,0.000170,0.000105
AMZN,0.000271,0.000517,0.000302,0.000124,0.000095
GOOGL,0.000269,0.000302,0.000419,0.000171,0.000086
JPM,0.000170,0.000124,0.000171,0.000426,0.000071
WMT,0.000105,0.000095,0.000086,0.000071,0.000204


In [6]:
sigma_inv = np.linalg.inv(sigma)
np.round(pd.DataFrame(sigma_inv),2) # matriz de covarianza inversa

,0,1,2,3,4
0,5134.21,-1211.03,-1905.45,-769.54,-997.35
1,-1211.03,3688.34,-1898.19,237.85,-386.31
2,-1905.45,-1898.19,5332.03,-809.29,-100.74
3,-769.54,237.85,-809.29,2980.78,-416.99
4,-997.35,-386.31,-100.74,-416.99,5795.57


In [7]:
ones = np.ones(len(mu))
ones # vector de unos

array([1., 1., 1., 1., 1.])

In [8]:
w = (sigma_inv.dot(ones))/(ones.T.dot(sigma_inv).dot(ones))
w # vector de pesos para portafolio de mínima varianza

array([0.03909055, 0.06711463, 0.09636506, 0.19056352, 0.60686624])

In [9]:
w_df = pd.DataFrame(w, index=closes.columns, columns=['w'])
w_df.T # pesos en DataFrame

Ticker,AAPL,AMZN,GOOGL,JPM,WMT
w,0.039091,0.067115,0.096365,0.190564,0.606866


In [19]:
rp = w.T.dot(mu)
rp # rendimiento esperado del portafolio de mínima varianza diario

0.0008450051383630455

In [11]:
mu

Ticker
AAPL     0.001136
AMZN     0.000852
GOOGL    0.000925
JPM      0.000789
WMT      0.000830
dtype: float64

## Segundo inciso que pide elegir activos

In [12]:
U = np.array([mu, ones]).T
U

array([[1.13554115e-03, 1.00000000e+00],
       [8.51879336e-04, 1.00000000e+00],
       [9.24837214e-04, 1.00000000e+00],
       [7.89105487e-04, 1.00000000e+00],
       [8.30406931e-04, 1.00000000e+00]])

In [13]:
M = U.T.dot(sigma_inv).dot(U)
pd.DataFrame(M)

,0,1
0,0.005005,5.422285
1,5.422285,6416.866046


In [14]:
M_inv = np.linalg.inv(M)
pd.DataFrame(M_inv)

,0,1
0,2364.584371,-1.998086
1,-1.998086,0.001844


In [15]:
u = np.array([[rp],
              [1]])
u

array([[8.45005138e-04],
       [1.00000000e+00]])

In [16]:
w2 = sigma_inv.dot(U).dot(M_inv).dot(u)
w2

array([[0.03909055],
       [0.06711463],
       [0.09636506],
       [0.19056352],
       [0.60686624]])

In [17]:
w2_df = pd.DataFrame(w2, index=closes.columns, columns=['w'])
w2_df.T # pesos en DataFrame

Ticker,AAPL,AMZN,GOOGL,JPM,WMT
w,0.039091,0.067115,0.096365,0.190564,0.606866
